In [1]:
import requests
import os
from osgeo import gdal, osr
import pandas as pd
import json
import datetime
import time
import zipfile
from pathlib import Path
import shutil

In [3]:
API_KEY_STAGING = ''
API_KEY = ''

HEADERS = {
            "username": '__key__',
            "password": API_KEY,
            "Content-Type": "application/json",
}

DATA_HEADERS = {
            "username": '__key__',
            "password": API_KEY,
}


RASTER_SOURCES_ENDPOINT = 'https://demo.lizard.net/api/v4/rastersources/'
RASTER_SOURCES_DATA_ENDPOINT = 'https://demo.lizard.net/api/v4/rastersources/{uuid}/data/'
RASTER_LAYERS_ENDPOINT = 'https://demo.lizard.net/api/v4/rasters/'
LAYERCOLLECTIONS_ENDPOINT = 'https://demo.lizard.net/api/v4/layercollections/'
ORGANISATIONS_ENDPOINT = 'https://demo.lizard.net/api/v4/organisations/'

NENS_ORGANISATION_STAGING = '61f5a464-c350-44c1-9bc7-d4b42d7f58cb'

MAXWAITTIME = 3*60*60

In [4]:
def asc_to_tif(asc_fn, tif_fn):
    drv = gdal.GetDriverByName('GTiff')
    ds_in = gdal.Open(asc_fn)
    ds_out = drv.CreateCopy(tif_fn, ds_in)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(28992)
    ds_out.SetProjection(srs.ExportToWkt())
    ds_out.GetRasterBand(1).SetNoDataValue(-999.9990)
    ds_in = None
    ds_out = None
    
def set_crs_for_tif(tif_fn):
    drv = gdal.GetDriverByName('GTiff')
    ds_out = gdal.Open(tif_fn, 1)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(28992)
    ds_out.SetProjection(srs.ExportToWkt())
    ds_out = None
 
def post_raster_source(name, description, organisation_uuid, temporal, interval=None, headers=HEADERS):    

    if temporal:
        raster_source =  {'name': name,
                          'description': description,
                          'organisation' : organisation_uuid,
                          'temporal': temporal,
                          'interval': interval}
    else:
        raster_source =  {'name': name,
                          'description': description,
                          'organisation' : organisation_uuid,
                          'temporal': temporal}

        
    r = requests.post(RASTER_SOURCES_ENDPOINT, json.dumps(raster_source), headers=headers)
    
    return r

def post_raster_layer(name, description, organisation_uuid, observation_type, source_uuid, style, layer_collection_slug, headers=HEADERS):
    raster_layer =  {'name': name,
                  'description': description,
                 'organisation' : organisation_uuid,
                 'observation_type': observation_type,
                 'source': {
                        "name": "RasterStoreSource",
                        "graph": {
                            "RasterStoreSource": [
                                "lizard_nxt.blocks.LizardRasterSource",
                                source_uuid
                            ]
                  }},
                 'options': {"styles": style},
                 'layer_collections': [layer_collection_slug]
    }

    r = requests.post(RASTER_LAYERS_ENDPOINT, json.dumps(raster_layer), headers=headers)
                     
    return r

def get_rastersource_uuid_scenario(scenario_id, variable, headers=HEADERS):
    params = {'name__endswith': 'ID:{}'.format(scenario_id),
              'name__icontains': variable}
            
    r = requests.get('https://demo.lizard.net/api/v4/rastersources', params=params, headers=headers)
    r.raise_for_status()
    return r.json()['results'][0]['uuid']

def get_rasterlayer_uuid_scenario(scenario_id, variable, headers=HEADERS):
    params = {'name__endswith': 'ID:{}'.format(scenario_id),
              'name__icontains': variable}
            
    r = requests.get('https://demo.lizard.net/api/v4/rasters', params=params, headers=headers)
    r.raise_for_status()
    return r.json()['results'][0]['uuid']

def delete_raster_with_scenario_id_and_variable(scenario_id, variable, headers=HEADERS):
    
    params = {'name__endswith': 'ID:{}'.format(scenario_id),
              'name__icontains': variable}
            
    r = requests.get('https://demo.lizard.net/api/v4/rasters', params=params, headers=headers)
    r.raise_for_status()

    if r.json()['count'] > 0:
        
        for result in r.json()['results']:
            raster_layer_url = result['url']
            r_layer_delete = requests.delete(raster_layer_url, headers=headers)
            r_layer_delete.raise_for_status()
            print('Removed rasterlayer {}...'.format(raster_layer_url))

    r = requests.get('https://demo.lizard.net/api/v4/rastersources', params=params, headers=headers)
    r.raise_for_status()
    
    if r.json()['count'] > 0:
        for result in r.json()['results']:
            raster_source_url = result['url']
            r_source_delete = requests.delete(raster_source_url, headers=headers)
            r_source_delete.raise_for_status()
            print('Removed rastersource {}...'.format(raster_source_url))


def delete_organisation_rasters(organisation_uuid):
    
    params = {'organisation__uuid': organisation_uuid}
    r = requests.get('https://demo.lizard.net/api/v4/rasters', params=params)
    r.raise_for_status()
    
    rasters
    
    
def create_or_return_layer_collection(slug, organisation_uuid, headers=HEADERS):

    params = {'slug':slug}
    r = requests.get(LAYERCOLLECTIONS_ENDPOINT, params=params, headers=headers)
    r.raise_for_status()
    
    if len(r.json()['results']) > 0:
        collection = r.json()['results'][0]
        return collection['slug']
    else:    
        collection =  {'slug': slug,
                       'organisation' : organisation_uuid}

        r = requests.post(LAYERCOLLECTIONS_ENDPOINT, json.dumps(collection), headers=headers)
        return r.json()['slug']
    
def search_organisation_by_name(organisation_name, headers=HEADERS):
    
    full_name = 'LDO - ' + organisation_name
    params = {'name':full_name}
    r = requests.get(ORGANISATIONS_ENDPOINT, params=params, headers=headers)
    r.raise_for_status()
    
    if r.json()['count'] > 0:
        result = r.json()['results'][0]
        uuid = result['uuid']
        return uuid
    else:
        return None    
    
def post_data_to_source(source_uuid, file):                 

    files = {"file": open(file, "rb")}
    url = RASTER_SOURCES_DATA_ENDPOINT.format(uuid=source_uuid)
    r = requests.post(url=url, files=files, headers={'username': '__key__','password': API_KEY})
    
    return r
                    

def upload_rasterperiod(url, df):
    """
    Function for uploading a period of rasters.
    Uploads the next one after the previous one succeeds, as the
    Lizard API often crashes otherwise. 
    Inputs are the url to upload (with uuid and /data/) to and
    a pandas dataframe. This df should have pandas datetime as index, and
    a column named file which contains the file path relative to cwd.
    All other columns are returned except columns with keywords task, status and waittime. 
    """
    
    upload_df = df.sort_index()
    for index, row in upload_df.iterrows():
        file = {"file": open(row["file"], "rb")}
        data = {"timestamp": index.isoformat() + "Z"}
        print("Posting for {}".format(index))
        
        try:
            r = requests.post(url=url, data=data, files=file, headers=DATA_HEADERS)
            r.raise_for_status()
            upload_df.loc[index, "task"] = r.json()["url"]
        except:
            print("Error, file post for {} failed, {}".format(index, r.json()))
            print('Moving on with next timestep...')
            continue
        
        waittime = 0
        #Set first status manually
        upload_df.loc[index,"status"]="UNKNOWN"
        while (upload_df.loc[index,"status"] not in ["SUCCESS", "FAILURE"]) and (waittime <= MAXWAITTIME):
            waittime += 10
            time.sleep(10)
            try:
                r = requests.get(upload_df.loc[index, "task"])
                upload_df.loc[index, "status"] = r.json()["status"]
                upload_df.loc[index, "waittime"] = waittime
            except:
                break
        
        print("Moving on with status {}".format(upload_df.loc[index, "status"]))
    
    return upload_df

def str_to_int(string):
    try:
        integer = int(string)
    except:
        integer = 0
    
    return integer

#### Scenario's voor workshop

In [5]:
scenarionummers = [14034, 14033, 21007, 21008, 21009, 21010, 21011, 21012, 21014, 21013, 19632, 19633, 5196, 19391, 20677, 20680, 14014, 14039, 4908]
all_scenario_zip = r'Y:\T_Berends\w0232_liwo_ldo\ExportLIWO_24112021_1526.zip'
output_path = r'C:\Users\Emile.deBadts\Documents\Projecten\liwo_data_posten\scenarios_workshop'

In [7]:
archive = zipfile.ZipFile(all_scenario_zip)

for file in archive.namelist():
    file_dir = file.split('/')[0]
    if str_to_int(file_dir) in scenarionummers:
        archive.extract(file, output_path)

#### LIWO scenario upload

Voor elk Dordrecht, zoek bijbehorende metadata file als die beschikbaar is. 
Vervolgens drie variabelen uploaden voor ieder scenario: max. wd (ASCI), max c (ASCI) en temporele waterdiepte


In [8]:
metadata_scenario_folders = os.listdir('./metadata')
export_scenario_folders = os.listdir('./eiland_van_dordrecht')
additional_metadata = pd.read_csv('scenarios.csv', sep='|')

upload_scenarios = [scenario for scenario in export_scenario_folders if scenario in metadata_scenario_folders]

In [43]:
# Scenarios voor workshop

In [16]:
upload_scenarios = os.listdir(output_path)

Convert .asci files to tiff

In [10]:
max_d_file = 'dm1maxd0.asc'
max_c_file = 'dm1maxc0.asc'
fls_inc_file = 'fls_h.inc'
export_files = [max_d_file, max_c_file, fls_inc_file]

In [11]:
complete_scenarios = []

for scenario in upload_scenarios:
    print(scenario)
    scenario_dir = os.path.join(output_path, scenario)
    scenario_files = os.listdir(scenario_dir)
    
    for folder in scenario_files:
        if folder.endswith('.zip'):
            zip_path = os.path.join(scenario_dir, folder)
            archive = zipfile.ZipFile(zip_path)
            for file in archive.namelist():
                if file in export_files:
                    archive.extract(file, scenario_dir)
  
    scenario_files = os.listdir(scenario_dir)

    if max_d_file in scenario_files:
        max_waterdepth = True
        print('   Create max waterdepth tiff')
        input_asci_fn = os.path.join(os.path.abspath(scenario_dir), 'dm1maxd0.asc')
        output_fn = os.path.join(scenario_dir, 'max_waterdepth.tif')
        asc_to_tif(asc_fn = input_asci_fn, tif_fn = output_fn)
    else:
        max_waterdepth = False

    if max_c_file in scenario_files:
        max_velocity = True
        print('   Create max velocity tiff')
        input_asci_fn = os.path.join(os.path.abspath(scenario_dir), 'dm1maxc0.asc')
        output_fn = os.path.join(scenario_dir, 'max_velocity.tif')
        asc_to_tif(asc_fn = input_asci_fn, tif_fn = output_fn)
    else:
        max_velocity = False
        
    # check fls files
#     temporal_wd_files = os.listdir(os.path.join(scenario_dir, 'fls'))
    if fls_inc_file in scenario_files:
        temporal_wd = True
    else:
        temporal_wd = False

    if all([temporal_wd, max_waterdepth, max_velocity]):
        complete_scenarios += [scenario]
        print('   Scenario complete!')
        


14014
   Create max waterdepth tiff
14033
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
14034
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
14039
   Create max waterdepth tiff
19391
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
19632
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
19633
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
20677
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
20680
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
21007
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
21008
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
21009
   Create max waterdepth tiff
   Create max velocity tiff
   Scenario complete!
21010
   Create max waterdepth tiff
   Create max velocity tiff
   S

### Create new raster sources and post data 

In [9]:
waterdepth_style = "Blues:0:2"
waterdepth_observation_type = 521


#### Scenario maximum waterdepth

In [8]:
posted_max_waterdepth = []

In [10]:
for scenario in scenarionummers:
    
    scenario = str(scenario)
    scenario_dir = os.path.join(output_path, scenario)
    waterdepth_file = os.path.join(scenario_dir, 'max_waterdepth.tif')

    if scenario not in posted_max_waterdepth and os.path.exists(waterdepth_file):
    
        print('Starting with scenario {}'.format(scenario))

        metadata = os.path.join(scenario_dir, 'metadata.csv') 
        metadata_df = pd.read_csv(metadata, sep= '|')

        # Data voor in name attribuut
        naam = metadata_df.loc['Scenarionaam', 'Scenario']
        overschrijding = metadata_df.loc['Overschrijdingsfrequentie', 'Scenario']
        naam_buitenwater = metadata_df.loc['Naam buitenwater', 'Scenario']
        buitenwatertype = metadata_df.loc['Buitenwatertype', 'Scenario']
        gebiedsnaam = metadata_df.loc['Gebiedsnaam', 'Scenario']
        naam_doorbraaklocatie = metadata_df.loc['Naam doorbraaklocatie', 'Scenario']
        
        name_string_format = f'{naam}_Variabele:MaximaleWaterdiepte_Overschrijdingsfrequentie:{overschrijding}_Naambuitenwater:{naam_buitenwater}_Buitenwatertype:{buitenwatertype}_Gebiedsnaam:{gebiedsnaam}_ID:{scenario}'
        metadata_json = json.dumps({row.name: row.Scenario for i,row in metadata_df.iterrows()})

        
        # Get the organisation UUID 
        #organisatie = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
        #organisation_uuid = search_organisation_by_name(organisatie)
        #if organisation_uuid is None:
        #    print('No organisation found for the current scenario')
        #    break
        organisation_uuid = '2e8ccd61-a0c3-473d-9bda-8b06db2bec35'
        
        print('Using organisation: {}'.format(organisation_uuid))
            
        # Get or post the layer collection
        #project = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
        #projectnaam = metadata_df.loc['Projectnaam', 'Scenario']
        #layer_collection_slug = r = create_or_return_layer_collection(slug=projectnaam, organisation_uuid=organisation_uuid)
        #print('Using layer collection: {}'.format(layer_collection_slug))
        layer_collection_slug = 'LDO_DEMO'
        
        # Max waterdepth 

        # Post the source 
        print('   Posting raster source for max waterdepth...')
        r_source_post = post_raster_source(name = name_string_format, 
                                           description = metadata_json, 
                                           organisation_uuid = organisation_uuid, 
                                           temporal = False)

        r_source_post.raise_for_status()

        # Post raster layer
        print('   Posting raster layer for max waterdepth...')
        r_layer_post = post_raster_layer(name = name_string_format, 
                                         description = metadata_json, 
                                         organisation_uuid = organisation_uuid, 
                                         observation_type = waterdepth_observation_type, 
                                         source_uuid = r_source_post.json()['uuid'], 
                                         style = waterdepth_style,
                                         layer_collection_slug=layer_collection_slug)

        r_layer_post.raise_for_status()

        # Post waterdepth tif
        print('   Posting data for max waterdepth...')
        r_data_post = post_data_to_source(source_uuid = r_source_post.json()['uuid'], file = waterdepth_file)

        r_data_post.raise_for_status()

        posted_max_waterdepth += [scenario]
        

Starting with scenario 14034
Using organisation: 2e8ccd61-a0c3-473d-9bda-8b06db2bec35
   Posting raster source for max waterdepth...
   Posting raster layer for max waterdepth...
   Posting data for max waterdepth...
Starting with scenario 14033
Using organisation: 2e8ccd61-a0c3-473d-9bda-8b06db2bec35
   Posting raster source for max waterdepth...
   Posting raster layer for max waterdepth...
   Posting data for max waterdepth...
Starting with scenario 21007
Using organisation: 2e8ccd61-a0c3-473d-9bda-8b06db2bec35
   Posting raster source for max waterdepth...
   Posting raster layer for max waterdepth...
   Posting data for max waterdepth...
Starting with scenario 21008
Using organisation: 2e8ccd61-a0c3-473d-9bda-8b06db2bec35
   Posting raster source for max waterdepth...
   Posting raster layer for max waterdepth...
   Posting data for max waterdepth...
Starting with scenario 21009
Using organisation: 2e8ccd61-a0c3-473d-9bda-8b06db2bec35
   Posting raster source for max waterdepth...

### Scenario maximum velocity

In [69]:
posted_flow_velocity = []

In [70]:
max_velocity_style = "Oranges:0:2"
max_velocity_observation_type = 969

In [71]:
for scenario in complete_scenarios:
    
    scenario_dir = os.path.join(output_path, scenario)
    velocity_file = os.path.join(scenario_dir, 'max_velocity.tif')
    
    if scenario not in posted_flow_velocity and os.path.exists(velocity_file):
    
        print('Starting with scenario {}'.format(scenario))

        metadata = os.path.join(scenario_dir, 'metadata.csv') 
        metadata_df = pd.read_csv(metadata, sep= '|')

        naam = metadata_df.loc['Scenarionaam', 'Scenario']
        variabele = 'MaximaleStroomsnelheid'
        overschrijding = metadata_df.loc['Overschrijdingsfrequentie', 'Scenario']
        naam_buitenwater = metadata_df.loc['Naam buitenwater', 'Scenario']
        buitenwatertype = metadata_df.loc['Buitenwatertype', 'Scenario']
        gebiedsnaam = metadata_df.loc['Gebiedsnaam', 'Scenario']
        naam_doorbraaklocatie = metadata_df.loc['Naam doorbraaklocatie', 'Scenario']

        name_string_format = f'{naam}_Variabele:{variabele}_Overschrijdingsfrequentie:{overschrijding}_Naambuitenwater:{naam_buitenwater}_Buitenwatertype:{buitenwatertype}_Gebiedsnaam:{gebiedsnaam}_ID:{scenario}'
        metadata_json = json.dumps({row.name: row.Scenario for i,row in metadata_df.iterrows()})

        # Get the organisation UUID 
        organisatie = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
        organisation_uuid = search_organisation_by_name(organisatie)
        if organisation_uuid is None:
            print('No organisation found for the current scenario')
            break
        
        print('Using organisation: {}'.format(organisatie))
            
        # Get or post the layer collection
        project = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
        projectnaam = metadata_df.loc['Projectnaam', 'Scenario']
        layer_collection_slug = r = create_or_return_layer_collection(slug=projectnaam, organisation_uuid=organisation_uuid)
        print('Using layer collection: {}'.format(layer_collection_slug))

        
        # Max waterdepth 

        # Post the source 
        print('   Posting raster source for max flow velocity...')
        r_source_post = post_raster_source(name = name_string_format, 
                                           description = metadata_json, 
                                           organisation_uuid = organisation_uuid, 
                                           temporal = False)

        r_source_post.raise_for_status()

        # Post raster layer
        print('   Posting raster layer for max flow velocity...')
        r_layer_post = post_raster_layer(name = name_string_format, 
                                         description = metadata_json, 
                                         organisation_uuid = organisation_uuid, 
                                         observation_type = max_velocity_observation_type, 
                                         source_uuid = r_source_post.json()['uuid'], 
                                         style = max_velocity_style,
                                         layer_collection_slug=layer_collection_slug)

        r_layer_post.raise_for_status()

        # Post waterdepth tif
        print('   Posting data for max flow velocity...')
        r_data_post = post_data_to_source(source_uuid = r_source_post.json()['uuid'], file = velocity_file)

        r_data_post.raise_for_status()

        posted_flow_velocity += [scenario]

Starting with scenario 14033
Using organisation: Provincie Zuid-Holland
Using layer collection: Gebiedsontwikeling Westergouwe
   Posting raster source for max flow velocity...
   Posting raster layer for max flow velocity...
   Posting data for max flow velocity...
Starting with scenario 14034
Using organisation: Provincie Zuid-Holland
Using layer collection: Gebiedsontwikeling Nieuwerkerk Noord
   Posting raster source for max flow velocity...
   Posting raster layer for max flow velocity...
   Posting data for max flow velocity...
Starting with scenario 19391
Using organisation: Provincie Noord-Brabant
Using layer collection: LIWO Noord-Brabant
   Posting raster source for max flow velocity...
   Posting raster layer for max flow velocity...
   Posting data for max flow velocity...
Starting with scenario 19632
Using organisation: Provincie Zuid-Holland
Using layer collection: Nut en noodzaak compartimenteringskeringen
   Posting raster source for max flow velocity...
   Posting rast

### Scenario temporal waterdepth

In [125]:
posted_temporal_waterdepth_data = []
temporal_waterdepth_source_uuids = {}
temporal_waterdepth_layer_uuids = {}

In [126]:
for scenario in complete_scenarios:

    print('Starting with scenario {}'.format(scenario))

    scenario_dir = os.path.join(output_path, scenario)
    metadata = os.path.join(scenario_dir, 'metadata.csv') 
    metadata_df = pd.read_csv(metadata, sep= '|')

    naam = metadata_df.loc['Scenarionaam', 'Scenario']
    variabele = 'TemporeleWaterdiepte'
    overschrijding = metadata_df.loc['Overschrijdingsfrequentie', 'Scenario']
    naam_buitenwater = metadata_df.loc['Naam buitenwater', 'Scenario']
    buitenwatertype = metadata_df.loc['Buitenwatertype', 'Scenario']
    gebiedsnaam = metadata_df.loc['Gebiedsnaam', 'Scenario']
    naam_doorbraaklocatie = metadata_df.loc['Naam doorbraaklocatie', 'Scenario']

    name_string_format = f'{naam}_Variabele:{variabele}_Overschrijdingsfrequentie:{overschrijding}_Naambuitenwater:{naam_buitenwater}_Buitenwatertype:{buitenwatertype}_Gebiedsnaam:{gebiedsnaam}_ID:{scenario}'
    metadata_json = json.dumps({row.name: row.Scenario for i,row in metadata_df.iterrows()})

    # Get the organisation UUID 
    organisatie = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
    organisation_uuid = search_organisation_by_name(organisatie)
    if organisation_uuid is None:
        print('No organisation found for the current scenario')
        break

    print('Using organisation: {}'.format(organisatie))

    # Get or post the layer collection
    project = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
    projectnaam = metadata_df.loc['Projectnaam', 'Scenario']
    layer_collection_slug = r = create_or_return_layer_collection(slug=projectnaam, organisation_uuid=organisation_uuid)
    print('Using layer collection: {}'.format(layer_collection_slug))

    # Post the source 

    if scenario not in temporal_waterdepth_source_uuids.keys():

        print('   Posting raster source for max waterdepth...')
        r_source_post = post_raster_source(name = name_string_format, 
                                           description = metadata_json, 
                                           organisation_uuid = organisation_uuid, 
                                           temporal = True,
                                           interval = '01:00:00')

        r_source_post.raise_for_status()
        temporal_waterdepth_source_uuids[scenario] = r_source_post.json()['uuid']


    if scenario not in temporal_waterdepth_layer_uuids.keys():
        # Post raster layer
        print('   Posting raster layer for max waterdepth...')
        r_layer_post = post_raster_layer(name = name_string_format, 
                                         description = metadata_json, 
                                         organisation_uuid = organisation_uuid, 
                                         observation_type = waterdepth_observation_type, 
                                         source_uuid = temporal_waterdepth_source_uuids[scenario], 
                                         style = waterdepth_style,
                                         layer_collection_slug=layer_collection_slug)

        r_layer_post.raise_for_status()
        temporal_waterdepth_layer_uuids[scenario] = r_layer_post.json()['uuid']

    # Post all waterdepths
    
    if scenario not in posted_temporal_waterdepth_data:
        print('   Posting data to the SOURCE :O')
    
        # Construct file dataframe with datetime
        start_simulatie = metadata_df.loc['Start simulatie', 'Scenario']
        einde_simulatie = metadata_df.loc['Einde simulatie', 'Scenario']
        
        try:
            start_simulatie_datetime = datetime.datetime.strptime(start_simulatie, "%d/%m/%Y %H:%M")
            eind_simulatie_datetime = datetime.datetime.strptime(einde_simulatie, "%d/%m/%Y %H:%M")
        except:
            print('No compatible datetime string')
        temporal_files = os.listdir(os.path.join(scenario_dir, 'fls'))
        temporal_wd_files = [os.path.join(scenario_dir, 'fls', file) for file in temporal_files if file.startswith('dataset') and file.endswith('.tiff')]

        for file in temporal_wd_files:
            set_crs_for_tif(file)

        datetime_range = [start_simulatie_datetime + datetime.timedelta(hours=x) for x in range(len(temporal_wd_files))]

        file_df = pd.DataFrame({'datetime':datetime_range, 'file':temporal_wd_files})
        file_df.set_index(file_df.datetime, inplace=True)

        url = RASTER_SOURCES_DATA_ENDPOINT.format(uuid=temporal_waterdepth_source_uuids[scenario])

        upload_df = upload_rasterperiod(url=url, df=file_df)        
        posted_temporal_waterdepth_data += [scenario]

Starting with scenario 14033
Using organisation: Provincie Zuid-Holland
Using layer collection: Gebiedsontwikeling Westergouwe
   Posting raster source for max waterdepth...
   Posting raster layer for max waterdepth...
   Posting data to the SOURCE :O


ValueError: time data '33239.0' does not match format '%d/%m/%Y %H:%M'

### Remove raster layer and source by ID and variable

In [7]:
variables = ['MaximaleWaterdiepte', 'MaximaleStroomsnelheid']
for scenario in scenarionummers:
    for variable in variables:
        scenario = str(scenario)
        delete_raster_with_scenario_id_and_variable(scenario_id=scenario, variable=variable)

KeyboardInterrupt: 

### Testing temporal waterdepth posting

In [5]:
scenario = '19749'
uuid_temporal_wd = get_rastersource_uuid_scenario(scenario, 'TemporeleWaterdiepte')

In [8]:
scenario_dir = os.path.join('./eiland_van_dordrecht/', scenario)
metadata = os.path.join('./metadata', scenario,'metadata.csv') 
metadata_df = pd.read_csv(metadata, sep= '|')

naam = metadata_df.loc['Scenarionaam', 'Scenario']
variabele = 'TemporeleWaterdiepte'
overschrijding = metadata_df.loc['Overschrijdingsfrequentie', 'Scenario']
naam_buitenwater = metadata_df.loc['Naam buitenwater', 'Scenario']
buitenwatertype = metadata_df.loc['Buitenwatertype', 'Scenario']
gebiedsnaam = metadata_df.loc['Gebiedsnaam', 'Scenario']
naam_doorbraaklocatie = metadata_df.loc['Naam doorbraaklocatie', 'Scenario']

name_string_format = f'{naam}_Variabele:{variabele}_Overschrijdingsfrequentie:{overschrijding}_Naambuitenwater:{naam_buitenwater}_Buitenwatertype:{buitenwatertype}_Gebiedsnaam:{gebiedsnaam}_ID:{scenario}'
metadata_json = json.dumps({row.name: row.Scenario for i,row in metadata_df.iterrows()})

# Get the organisation UUID 
organisatie = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
organisation_uuid = search_organisation_by_name(organisatie)
if organisation_uuid is None:
    print('No organisation found for the current scenario')
    exit

print('Using organisation: {}'.format(organisatie))

# Get or post the layer collection
# project = metadata_df.loc['Eigenaar overstromingsinformatie', 'Scenario']
# projectnaam = metadata_df.loc['Projectnaam', 'Scenario']
# layer_collection_slug = r = create_or_return_layer_collection(slug=projectnaam, organisation_uuid=organisation_uuid)
# print('Using layer collection: {}'.format(layer_collection_slug))

# # Post the source 

# print('   Posting raster source for max waterdepth...')
# r_source_post = post_raster_source(name = name_string_format, 
#                                    description = metadata_json, 
#                                    organisation_uuid = organisation_uuid, 
#                                    temporal = True,
#                                    interval = '01:00:00')

# r_source_post.raise_for_status()
# temporal_waterdepth_source_uuids[scenario] = r_source_post.json()['uuid']


# # Post raster layer
# print('   Posting raster layer for max waterdepth...')
# r_layer_post = post_raster_layer(name = name_string_format, 
#                                  description = metadata_json, 
#                                  organisation_uuid = organisation_uuid, 
#                                  observation_type = waterdepth_observation_type, 
#                                  source_uuid = temporal_waterdepth_source_uuids[scenario], 
#                                  style = waterdepth_style,
#                                  layer_collection_slug=layer_collection_slug)

# r_layer_post.raise_for_status()
# temporal_waterdepth_layer_uuids[scenario] = r_layer_post.json()['uuid']


# Construct file dataframe with datetime
start_simulatie = metadata_df.loc['Start simulatie', 'Scenario']
einde_simulatie = metadata_df.loc['Einde simulatie', 'Scenario']

start_simulatie_datetime = datetime.datetime.strptime(start_simulatie, "%d/%m/%Y %H:%M")
eind_simulatie_datetime = datetime.datetime.strptime(einde_simulatie, "%d/%m/%Y %H:%M")

print('Posting data...')
temporal_files = os.listdir(os.path.join(scenario_dir, 'fls'))
temporal_wd_files = [os.path.join(scenario_dir, 'fls', file) for file in temporal_files if file.startswith('dataset') and file.endswith('.tiff')]

for file in temporal_wd_files:
    set_crs_for_tif(file)

datetime_range = [start_simulatie_datetime + datetime.timedelta(hours=x) for x in range(len(temporal_wd_files))]

file_df = pd.DataFrame({'datetime':datetime_range, 'file':temporal_wd_files})
file_df.set_index(file_df.datetime, inplace=True)

url = RASTER_SOURCES_DATA_ENDPOINT.format(uuid=uuid_temporal_wd)

upload_df = upload_rasterperiod(url=url, df=file_df)        
posted_temporal_waterdepth_data += [scenario]

Using organisation: Provincie Zuid-Holland
Posting data...
Posting for 1991-01-06 00:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 01:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 02:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 03:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 04:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 05:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 06:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 07:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 08:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 09:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 10:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 11:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 12:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 13:00:00
Moving on with status SUCCESS
Posting for 1991-01-06 14:00:00
Moving on with status SUCCESS
Posting for

Moving on with status SUCCESS
Posting for 1991-01-11 12:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 13:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 14:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 15:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 16:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 17:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 18:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 19:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 20:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 21:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 22:00:00
Moving on with status SUCCESS
Posting for 1991-01-11 23:00:00
Moving on with status SUCCESS
Posting for 1991-01-12 00:00:00
Moving on with status SUCCESS
Posting for 1991-01-12 01:00:00
Moving on with status SUCCESS
Posting for 1991-01-12 02:00:00
Moving on with status SUCCESS
Posting for 1991-01-12 03:00:00
Moving o

Moving on with status SUCCESS
Posting for 1991-01-17 01:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 02:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 03:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 04:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 05:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 06:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 07:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 08:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 09:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 10:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 11:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 12:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 13:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 14:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 15:00:00
Moving on with status SUCCESS
Posting for 1991-01-17 16:00:00
Moving o

Moving on with status SUCCESS
Posting for 1991-01-22 14:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 15:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 16:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 17:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 18:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 19:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 20:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 21:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 22:00:00
Moving on with status SUCCESS
Posting for 1991-01-22 23:00:00
Moving on with status SUCCESS


NameError: name 'posted_temporal_waterdepth_data' is not defined

In [29]:
variables = ['MaximaleWaterdiepte', 'MaximaleStroomsnelheid']

rasterlayer_url = 'https://demo.lizard.net/api/v4/rasters/{}'
rastersource_url = 'https://demo.lizard.net/api/v4/rastersources/{}'

# Patch organisations
for scenario in scenario_select:
    scenario = str(scenario)
    for variable in variables:
        
        print('...',variable)
        
        try:
            rastersource_uuid = get_rastersource_uuid_scenario(scenario_id=scenario, variable=variable)
            rasterlayer_uuid = get_rasterlayer_uuid_scenario(scenario_id=scenario, variable=variable)
        except:
            print('Not found')
            continue
        
        r_rl = requests.patch(rasterlayer_url.format(rasterlayer_uuid), 
                              data=json.dumps({'organisation':'2e8ccd61-a0c3-473d-9bda-8b06db2bec35'}),
                              headers=HEADERS)
        r_rl.raise_for_status()
        
        r_rs = requests.patch(rastersource_url.format(rastersource_uuid), 
                              data=json.dumps({'organisation':'2e8ccd61-a0c3-473d-9bda-8b06db2bec35'}),
                              headers=HEADERS)
        r_rs.raise_for_status()
        


21014
... MaximaleWaterdiepte
... MaximaleStroomsnelheid


### Data naar staging posten

In [26]:
# Patch raster layer with Group geoblock with statistic
layer_url = 'https://demo.lizard.net/api/v4/rasters/846fde77-19bc-496a-8337-c77c9e9543e9/'

with open('group_with_statistic_graph.json') as f:
    group_geoblock = json.loads(f.read())

In [27]:
for i, scenario in enumerate(scenarionummers):
    scenario = str(scenario)
    source = get_rastersource_uuid_scenario(scenario_id=scenario, variable='MaximaleWaterdiepte')
    rasterblock = ['lizard_nxt.blocks.LizardRasterSource', source]
    rasterblock_name = 'waterdepth_' + str(i)
    group_geoblock['graph'][rasterblock_name] = rasterblock
    group_geoblock['graph']['group'].append(rasterblock_name)

In [28]:
group_geoblock

{'name': 'mask',
 'graph': {'group': ['dask_geomodeling.raster.reduction.Max',
   'waterdepth_0',
   'waterdepth_1',
   'waterdepth_2',
   'waterdepth_3',
   'waterdepth_4',
   'waterdepth_5',
   'waterdepth_6',
   'waterdepth_7',
   'waterdepth_8',
   'waterdepth_9',
   'waterdepth_10',
   'waterdepth_11',
   'waterdepth_12',
   'waterdepth_13',
   'waterdepth_14',
   'waterdepth_15',
   'waterdepth_16',
   'waterdepth_17',
   'waterdepth_18'],
  'mask': ['dask_geomodeling.raster.misc.MaskBelow', 'group', 0.01],
  'waterdepth_0': ['lizard_nxt.blocks.LizardRasterSource',
   '024c8fb2-b155-42ee-a7db-0dca0d1e1bfe'],
  'waterdepth_1': ['lizard_nxt.blocks.LizardRasterSource',
   '7fc7196f-8bf1-4ddf-8dd9-6a9b3422a86b'],
  'waterdepth_2': ['lizard_nxt.blocks.LizardRasterSource',
   'c8c460aa-a487-4375-b84a-0802213fed6e'],
  'waterdepth_3': ['lizard_nxt.blocks.LizardRasterSource',
   'e4ebb1dc-da20-4e60-9b98-4e263f54db7e'],
  'waterdepth_4': ['lizard_nxt.blocks.LizardRasterSource',
   'bc6358

In [17]:
r = requests.patch(layer_url, data=json.dumps({'source':group_geoblock}), headers=HEADERS)

In [24]:
json.dumps(group_geoblock)

'{"name": "mask", "graph": {"group": ["dask_geomodeling.raster.reduction.Max", "waterdepth_0", "waterdepth_1", "waterdepth_2", "waterdepth_3"], "mask": ["dask_geomodeling.raster.misc.MaskBelow", "group", 0.01], "waterdepth_0": ["lizard_nxt.blocks.LizardRasterSource", "024c8fb2-b155-42ee-a7db-0dca0d1e1bfe"], "waterdepth_1": ["lizard_nxt.blocks.LizardRasterSource", "7fc7196f-8bf1-4ddf-8dd9-6a9b3422a86b"], "waterdepth_2": ["lizard_nxt.blocks.LizardRasterSource", "c8c460aa-a487-4375-b84a-0802213fed6e"], "waterdepth_3": ["lizard_nxt.blocks.LizardRasterSource", "e4ebb1dc-da20-4e60-9b98-4e263f54db7e"]}}'